In [1]:
from custom_dataset import getDataset
import neural_net
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModel
import glob
import pandas as pd
import numpy as np
import preproc_tabular

target = [1,2,1,4,2,4,3,5,1,0] #TARGET FAKE
img_path = 'D:\\ORS\\Data\\ResizedImages\\'
tab_path = 'D:\\ORS\\Data\\prova.xlsx'

data, descriptions, references = preproc_tabular.get_tabular(img_path, tab_path)
print(descriptions)

d:\Programmi\Python3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Programmi\Python3.12\Lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


{'input_ids': tensor([[  101,  4440,  8684, 10250,  4371, 15908,   102,     0,     0,     0,
             0,     0,     0],
        [  101, 12170, 23947, 10250,  4371,  3609,  3796,   102,     0,     0,
             0,     0,     0],
        [  101, 12170, 23947, 10250,  4371,  3609,  3796,   102,     0,     0,
             0,     0,     0],
        [  101, 12170, 23947, 10250,  4371,  3609,  3796,   102,     0,     0,
             0,     0,     0],
        [  101, 12170, 23947, 10250,  4371,  1037, 19838,  5369,   102,     0,
             0,     0,     0],
        [  101, 12170, 23947, 10250,  4371,  1037, 19838,  5369,   102,     0,
             0,     0,     0],
        [  101, 12170, 23947, 10250,  4371,  1037, 19838,  5369,   102,     0,
             0,     0,     0],
        [  101, 12170, 23947, 10250,  4371,  1039,  1013, 20010, 15900,  3669,
         19857,  2080,   102],
        [  101, 12170, 23947, 10250,  4371,  1039,  1013, 20010, 15900,  3669,
         19857,  2080,   10